In [1]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("FEM.jl")
using .FEM

In [2]:
# Adatok
E = 2e5
ν = 0.3
thickness = 1
height = 100
base = 100
elemSize = 5 #22

approxOrder = 2
internalNodes = true
quadElements = false

problem = FEM.Problem("rectangle", type="PlaneStress", E=E, ν=ν, b=thickness)

Main.FEM.Problem("rectangle", "PlaneStress", 200000.0, 0.3, 7.85e-9, 1.0)

In [3]:
gmsh.open("rectangle.msh")

Info    : Reading 'rectangle.msh'...
Info    : 20 entities
Info    : 67 nodes
Info    : 11 elements
Info    : Done reading 'rectangle.msh'


In [4]:
bc1 = FEM.displacementConstraint("bottom", uy=0)
bc2 = FEM.displacementConstraint("left", ux=0)
ld1 = FEM.traction("top", fy=1)

("top", 0, 1, 1)

In [5]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.1-git-38a28aaaf
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240111
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [6]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [7]:
K = FEM.stiffnessMatrixPlaneStress(problem)

0×0 SparseMatrixCSC{Float64, Int64} with 0 stored entries

In [8]:
K, f = FEM.applyBoundaryConditions!(problem, K, [bc1, bc2], [ld1])

Error   : Physical name 'top' does not exist


ErrorException: Physical name 'top' does not exist

In [9]:
#det(K)

In [10]:
q = FEM.solveDisplacement(K, f)

UndefVarError: UndefVarError: `f` not defined

In [11]:
S = FEM.solveStressPlaneStress(problem, q)

UndefVarError: UndefVarError: `q` not defined

In [12]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [13]:
u = FEM.showResultUvec(problem, q, name="uvec", visible=false)
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
sx = FEM.showResultSX(problem, S, name="σx", visible=false, smooth=false)
sy = FEM.showResultSY(problem, S, name="σy", visible=true, smooth=true)
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)

UndefVarError: UndefVarError: `q` not defined

In [14]:
pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
pp2 = gmsh.model.occ.addPoint(base / 4, height / 2, 0)
pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

pl1 = gmsh.model.occ.addLine(pp1, pp2)
pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


gmsh.model.occ.synchronize()

In [15]:
FEM.plotOnPath(problem, pl1, sy, 100, name="σred", visible=false);

UndefVarError: UndefVarError: `sy` not defined

In [16]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [17]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.1-git-38a28aaaf
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20240111
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [18]:
gmsh.finalize()